In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [ ]:
import os

DIR = os.path.abspath("..\\..\\")
DATA_DIR = DIR + "\\input\\qmdata0819"

DATA_DIR

## 读取数据集

In [ ]:
read_all_csv = open(DATA_DIR + "\\all.csv", "r" , encoding="utf8")

data_ = []
for line in read_all_csv:
    line = line.strip()
    time_, item_ = line.split(',')
    data_.append([time_, item_])

# 第一行是表格的标题栏，pop掉
data_.pop(0)

为 data_ 增加第三维内容。
第三维内容是时间戳，来自每条记录的时间文本

In [ ]:
import time

for each in data_:
    each.append(int(time.mktime(time.strptime(each[0], "%Y-%m-%d %H:%M:%S"))))

len(data_)

根据时间戳，分析数据的段落。前后2条数据的时间戳间隔不为58的，都视为段落分割点。

In [ ]:
segment = []
all_segs = []
for i in range(len(data_)):
    segment.append(data_[i])
    if i<len(data_)-1 and i > 0 and data_[i+1][2] - data_[i][2] != 58:
        all_segs.append(segment)
        segment = []

all_segs.append(segment)

对于每一个段落，读取它的第0 - 20条数据，作为新的数据段落的起点。把它对应的第21条数据作为相应的结果。依次逐条推进，形成新的数据集段落列表 all_rows。

In [ ]:
row_ = []
all_rows = []
for each in all_segs:
    for i in range(20,len(each)-1):
        list_ = each[i-20:i]
        row_ = [list_, each[i]]
        all_rows.append(row_)

len(all_rows)

为 all_rows 里的每一条数据，增加一项统计数据，计算这一条数据里的20条历史记录，各物品对应的统计量。

In [ ]:
item_names = ["钢琴", "小提琴", "吉他", "贝斯", "架子鼓", "竖琴", "萨克斯风", "圆号"]
def count_items(a_list:list):
    items = []
    # each 是每个回合的数据
    # each[1]是名字
    for each in a_list:
        items.append(each[1])
    
    count_result = []
    for n in item_names:
        count_result.append(items.count(n))
    
    # count_result是每一行数据的20个回合里，每个物品数量的统计
    return count_result

# each 是20个回合的出货数据 + 第21回合的出货数据
# each[0] 是20回合的结果
history = []
for each in all_rows:
    history.append(count_items(each[0]))

len(history)
history[-1]

In [ ]:
merge_data = []
for i in range(len(history)):
    merge_data.append([all_rows[i][0], all_rows[i][1], history[i]])

merge_data[-1]

## 构建数据集
merge_data 里的 each ，each[0]里的每个 item，抽取 item[1]，即物品的名字

In [ ]:
import numpy as np
items_name_20 = []
item_result = ''
count = []
data_for_pd = []
for each in merge_data:
    item_result = each[1][1]
    count = each[2]
    if item_result in item_names[:4]:
        if count[item_names.index(item_result)] > np.mean(count[:4]):
            result = '高'
        elif count[item_names.index(item_result)] < np.mean(count[:4]):
            result = '低'
        else:
            result = '平'
    elif item_result in item_names[-4:]:
        if count[item_names.index(item_result)] > np.mean(count[-4:]):
            result = '高'
        elif count[item_names.index(item_result)] < np.mean(count[-4:]):
            result = '低'
        else:
            result = '平'
    for item in each[0]:
        items_name_20.append(item[1])
    high_low = []
    for item in items_name_20:
        if item in item_names[:4]:
            if count[item_names.index(item)] > np.mean(count[:4]):
                high_low.append('高')
            elif count[item_names.index(item)] < np.mean(count[:4]):
                high_low.append('低')
            else:
                high_low.append('平')
        elif item in item_names[-4:]:
            if count[item_names.index(item)] > np.mean(count[-4:]):
                high_low.append('高')
            elif count[item_names.index(item)] < np.mean(count[-4:]):
                high_low.append('低')
            else:
                high_low.append('平')
    data_for_pd.append([items_name_20,high_low, count, item_result, result])
    items_name_20 = []


data_for_pd[-1]

In [ ]:
a_1_4 = []
for each in data_for_pd:
    a_1_4.append(each[1]+[each[4]])

a_1_4[-2:]

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [ ]:
pd_1_4 = pd.DataFrame(a_1_4,columns=['col_1','col_2','col_3','col_4','col_5','col_6','col_7','col_8','col_9','col_10','col_11','col_12','col_13','col_14','col_15','col_16','col_17','col_18','col_19','col_20','result',])

pd_1_4.head()

In [ ]:
pd_1_4.isnull().sum()

In [ ]:
y = pd_1_4['result']

X = pd_1_4.drop(['result'], axis=1)

print(f'X: {X.shape}')

In [ ]:
from sklearn.model_selection import  train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)

print(f'X_train: {X_train.shape}')
print(f'X_test: {X_test.shape}')
print(f'y_train: {y_train.shape}')
print(f'y_test: {y_test.shape}')

## 超参数

In [ ]:
# 随机森林树的数量
n_estimators = [int(x) for x in np.linspace(start=10, stop=80, num=10)]
# 每个分割集上的最大特征数
max_featues = ['auto', 'sqrt']
# 最大树层级数
max_depth = [2,4]
# 最小样本数
min_samples_split = [2,5]
min_samples_leaf = [1,2]
bootstrap = [True, False]


In [ ]:
param_grid = {
    'n_estimators': n_estimators,
    'max_features': max_featues,
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'bootstrap': bootstrap
}
print(param_grid)

In [ ]:
rf_Model = RandomForestClassifier()

In [ ]:
from sklearn.model_selection import GridSearchCV
rf_Grid = GridSearchCV(estimator=rf_Model, param_grid=param_grid, cv=10, verbose=2, n_jobs=4)

In [ ]:
rf_Grid.fit(X_train, y_train)